In [2]:
import io
import sys
PATH = "D:/OneDrive - Universidad Tecnológica de Bolívar/Apps\MachineLearning-ISCO-A14A/"
DIR_DATA = PATH + "data/"
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn import preprocessing
from logic.text_processing import TextProcessing

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
tp = TextProcessing()
nlp = spacy.load("es_core_news_sm")

In [5]:
data = pd.read_csv(DIR_DATA + 'tass2018_es_train.csv', sep=',')
data

,tweetid,user,content,date,lang,sentiment/polarity/value
0,768213876278165504,OnceBukowski,-Me caes muy bien \r\n-Tienes que jugar más pa...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\r\n\r\...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N
...,...,...,...,...,...,...
1003,814846333601320960,jdomperfects,Para mí mi mejor año fue el 2015,2016-12-30 14:51:19,es,P
1004,813731371076243461,GonzaloRoman_91,Hoy va a ser un dia muy largo...,2016-12-27 13:00:51,es,N
1005,818399956792905728,vaneftdallas,11:11 que no me encuentre mal,2017-01-09 10:12:09,es,NEU
1006,815715581878009858,carmen_ligero,Es que son muy bonitas jo,2017-01-02 00:25:24,es,P


In [6]:
data_test = [tp.transformer(row, stopwords=True) for row in tqdm(data['content'].tolist())]
len(data_test)

100%|██████████| 1008/1008 [02:31<00:00,  6.64it/s]


1008

In [7]:
def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1
    
    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [ ]:
df = co_occurrence(data_test, 2)
df